In [ ]:
# #logistic regression
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Subset, TensorDataset
# from sklearn.datasets import make_classification
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler
# import numpy as np
# import pandas as pd

# # Load the data
# data = pd.read_csv('pseudodata_præoperation.csv')

# #make pd to np
# X = data.to_numpy()
# y = np.random.choice([0, 1], size=len(data))

# # Generate a synthetic dataset
# X = StandardScaler().fit_transform(X)

# # Convert to PyTorch tensors
# X_tensor = torch.tensor(X, dtype=torch.float32)
# y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# # Create a dataset
# dataset = TensorDataset(X_tensor, y_tensor)

# # Split data into holdout set and remaining set
# indices = np.arange(len(dataset))
# np.random.seed(42)
# np.random.shuffle(indices)
# holdout_indices = indices[:500]
# remaining_indices = indices[500:]

# holdout_set = Subset(dataset, holdout_indices)
# remaining_set = Subset(dataset, remaining_indices)

# class_weights = torch.tensor([1/0.98, 1/0.02])

# # Define the logistic regression model with L2 regularization
# class LogisticRegressionModel(nn.Module):
#     def __init__(self, input_dim):
#         super(LogisticRegressionModel, self).__init__()
#         self.linear = nn.Linear(input_dim, 1)
    
#     def forward(self, x):
#         out = torch.sigmoid(self.linear(x))
#         return out

# input_dim = 575

# # Function to train and evaluate the model with L2 regularization
# def train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5):
#     model = LogisticRegressionModel(input_dim)
#     criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
#     optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    
#     # Training loop
#     for epoch in range(num_epochs):
#         model.train()
#         for images, labels in train_loader:
#             # Forward pass
#             outputs = model(images)
#             loss = criterion(outputs, labels)
            
#             # Backward pass and optimization
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
    
#     # Evaluate on validation set
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for images, labels in val_loader:
#             outputs = model(images)
#             predicted = (outputs > 0.5).float()
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
    
#     accuracy = correct / total
#     return accuracy, model

# # Outer 10-Fold Cross-Validation
# outer_kf = KFold(n_splits=10, shuffle=True, random_state=42)
# outer_accuracies = []
# best_model_params = None
# best_outer_accuracy = 0

# y_remaining = y_tensor[remaining_indices]

# for outer_train_index, outer_test_index in outer_kf.split(remaining_indices):
#     outer_train_subset = Subset(remaining_set, outer_train_index)
#     outer_test_subset = Subset(remaining_set, outer_test_index)
    
#     # Inner 10-Fold Cross-Validation
#     inner_kf = KFold(n_splits=10, shuffle=True, random_state=42)
#     inner_accuracies = []

#     for inner_train_index, inner_val_index in inner_kf.split(outer_train_index):
#         inner_train_indices = np.array(outer_train_index)[inner_train_index]
#         inner_val_indices = np.array(outer_train_index)[inner_val_index]
        
#         inner_train_subset = Subset(remaining_set, inner_train_indices)
#         inner_val_subset = Subset(remaining_set, inner_val_indices)
        
#         train_loader = DataLoader(inner_train_subset, batch_size=64, shuffle=True)
#         val_loader = DataLoader(inner_val_subset, batch_size=64, shuffle=False)
        
#         accuracy, model = train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5)
#         inner_accuracies.append(accuracy)
#         print(f'Inner Fold Accuracy: {accuracy * 100:.2f}%')

#     mean_inner_accuracy = np.mean(inner_accuracies)
#     print(f'Outer Fold Mean Inner Accuracy: {mean_inner_accuracy * 100:.2f}%')

#     train_loader = DataLoader(outer_train_subset, batch_size=64, shuffle=True)
#     test_loader = DataLoader(outer_test_subset, batch_size=64, shuffle=False)
    
#     outer_accuracy, outer_model = train_and_evaluate_model(train_loader, test_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5)
#     outer_accuracies.append(outer_accuracy)
#     print(f'Outer Fold Accuracy: {outer_accuracy * 100:.2f}%')

#     if outer_accuracy > best_outer_accuracy:
#         best_outer_accuracy = outer_accuracy
#         best_model_params = outer_model.state_dict()

# print(f'Mean Outer Accuracy: {np.mean(outer_accuracies) * 100:.2f}%')
# print('Best model parameters:', best_model_params)

# # Evaluate the final model on the holdout set
# final_model = LogisticRegressionModel(input_dim)
# final_model.load_state_dict(best_model_params)
# holdout_loader = DataLoader(holdout_set, batch_size=64, shuffle=False)

# # Evaluate on holdout set
# final_model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for images, labels in holdout_loader:
#         outputs = final_model(images)
#         predicted = (outputs > 0.5).float()
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# holdout_accuracy = correct / total
# print(f'Holdout Set Accuracy: {holdout_accuracy * 100:.2f}%')




In [ ]:
# from sklearn.metrics import roc_curve, auc, recall_score
# import matplotlib.pyplot as plt

# # Function to calculate AUC-ROC curve and recall score
# def plot_roc_curve_and_recall(model, loader, title="ROC Curve and Recall Score"):
#     model.eval()
#     y_true = []
#     y_scores = []

#     with torch.no_grad():
#         for images, labels in loader:
#             outputs = model(images)
#             y_true.extend(labels.numpy())
#             y_scores.extend(outputs.numpy())

#     fpr, tpr, thresholds = roc_curve(y_true, y_scores)
#     roc_auc = auc(fpr, tpr)
#     plt.figure()
#     lw = 2
#     plt.plot(fpr, tpr, color='darkorange',
#              lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
#     plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title(title)
#     plt.legend(loc="lower right")

#     # Calculate Recall Score
#     y_pred = (np.array(y_scores) > 0.5).astype(int)
#     recall = recall_score(y_true, y_pred)
#     print("Recall Score:", recall)

# # Define DataLoader for holdout set
# holdout_loader = holdout_log[1]

# # Load best model parameters
# final_model = models_log[1]

# # Plot ROC curve and calculate recall score
# plot_roc_curve_and_recall(final_model, holdout_loader)
# plt.show()



In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Subset, TensorDataset
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler
# import numpy as np
# import pandas as pd
# from sklearn.metrics import precision_score, recall_score, f1_score
# from sklearn.model_selection import train_test_split


# # Load the data
# data1 = pd.read_csv('pseudodata_præoperation.csv')
# data2 = pd.read_csv('pseudodata_præoperation.csv')
# data3 = pd.read_csv('pseudodata_præoperation.csv')
# data4 = pd.read_csv('pseudodata_præoperation.csv')
# data5 = pd.read_csv('pseudodata_præoperation.csv')
# data6 = pd.read_csv('pseudodata_præoperation.csv')

# base = data1
# phase1 = pd.concat([data1, data2], axis = 1)
# phase2 = pd.concat([data1, data2, data3], axis = 1)
# phase3 = pd.concat([data1, data2, data3, data4], axis = 1)
# phase4 = pd.concat([data1, data2, data3, data4, data5], axis = 1)
# phase5 = pd.concat([data1, data2, data3, data4, data5, data6], axis = 1)



# data = pd.read_csv('pseudodata_præoperation.csv')

# # Convert to numpy array and generate synthetic labels for demonstration
# X = data.to_numpy()
# y = np.random.choice([0, 1], size=len(data))

# # Standardize the data
# X = StandardScaler().fit_transform(X)

# # Convert to PyTorch tensors
# X_tensor = torch.tensor(X, dtype=torch.float32)
# y_tensor = torch.tensor(y, dtype=torch.float32)

# # Create a dataset
# dataset = TensorDataset(X_tensor, y_tensor)

# # Split data into holdout set and remaining set
# train_indices, holdout_indices = train_test_split(np.arange(len(dataset)), test_size=500, random_state=42, stratify=y_tensor.numpy())

# holdout_set = Subset(dataset, holdout_indices)
# remaining_set = Subset(dataset, train_indices)

# y_1_percent = np.count_nonzero(y_tensor) / len(y_tensor)
# y_0_percent = 1 - y_1_percent
# print(f"Class 1 percentage: {y_1_percent * 100:.2f}%")
# print(f"Class 0 percentage: {y_0_percent * 100:.2f}%")
# # Define class weights for BCEWithLogitsLoss
# class_weights = torch.tensor([1/y_0_percent, 1/y_1_percent])

# # Define the logistic regression model with L2 regularization
# class LogisticRegressionModel(nn.Module):
#     def __init__(self, input_dim):
#         super(LogisticRegressionModel, self).__init__()
#         self.linear = nn.Linear(input_dim, 1)
    
#     def forward(self, x):
#         out = torch.sigmoid(self.linear(x))
#         return out

# input_dim = data.shape[1]

# # Function to get target tensor from Subset
# def get_targets(subset):
#     return subset.dataset.tensors[1][subset.indices]

# # Get target tensors for holdout and remaining sets
# holdout_targets = get_targets(holdout_set)
# remaining_targets = get_targets(remaining_set)

# # Print class distributions
# print(f"Class distribution in holdout set: {np.unique(holdout_targets.numpy(), return_counts=True)}")
# print(f"Class distribution in remaining set: {np.unique(remaining_targets.numpy(), return_counts=True)}")

# weight_decay_values = np.logspace(-5, 1, 7)  # for example, try values from 1e-5 to 10

# # Function to train and evaluate the model with L2 regularization
# def train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5):
#     model = LogisticRegressionModel(input_dim)
#     criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
#     optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    
#     # Training loop
#     for epoch in range(num_epochs):
#         model.train()
#         for inputs, labels in train_loader:
#             # Forward pass
#             outputs = model(inputs)
#             loss = criterion(outputs.squeeze(), labels)
            
#             # Backward pass and optimization
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
    
#     # Evaluate on validation set
#     model.eval()
#     all_preds = []
#     all_labels = []
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             outputs = model(inputs)
#             predicted = (outputs > 0.5).float().squeeze()
#             all_preds.extend(predicted.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#     f1 = f1_score(all_labels, all_preds)
#     return f1, model

# outer_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# # Function to perform grid search over weight_decay values
# def grid_search(train_set, outer_kf, weight_decays, input_dim, num_epochs=10, lr=0.01):
#     best_weight_decay = None
#     best_outer_f1 = 0
#     y_remaining = y_tensor[train_indices].numpy()

#     for weight_decay in weight_decays:
#         outer_f1_scores = []
        
#         for outer_train_index, outer_test_index in outer_kf.split(np.zeros(len(train_set)), y_remaining):
            
#             outer_train_subset = Subset(train_set, outer_train_index)
#             outer_test_subset = Subset(train_set, outer_test_index)
            
#             # Inner 5-Fold Cross-Validation
#             inner_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#             inner_f1_scores = []
            
#             for inner_train_index, inner_val_index in inner_kf.split(np.zeros(len(outer_train_index)), y_remaining[outer_train_index]):
                
#                 inner_train_indices = np.array(outer_train_index)[inner_train_index]
#                 inner_val_indices = np.array(outer_train_index)[inner_val_index]
                
#                 inner_train_subset = Subset(train_set, inner_train_indices)
#                 inner_val_subset = Subset(train_set, inner_val_indices)
                
#                 train_loader = DataLoader(inner_train_subset, batch_size=64, shuffle=True)
#                 val_loader = DataLoader(inner_val_subset, batch_size=64, shuffle=False)
                
#                 f1, _ = train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs, lr, weight_decay)
#                 inner_f1_scores.append(f1)
                        
#             # Train on outer train subset and evaluate on outer test subset
#             train_loader = DataLoader(outer_train_subset, batch_size=64, shuffle=True)
#             test_loader = DataLoader(outer_test_subset, batch_size=64, shuffle=False)
            
#             outer_f1, _ = train_and_evaluate_model(train_loader, test_loader, input_dim, num_epochs, lr, weight_decay)
#             outer_f1_scores.append(outer_f1)
        
#         mean_outer_f1 = np.mean(outer_f1_scores)
#         print(f'Weight Decay: {weight_decay}, Mean Outer F1 Score: {mean_outer_f1:.4f}')
        
#         if mean_outer_f1 > best_outer_f1:
#             best_outer_f1 = mean_outer_f1
#             best_weight_decay = weight_decay
    
#     return best_weight_decay


# # Perform grid search over weight_decay values

# best_weight_decay = grid_search(remaining_set, outer_kf, weight_decay_values, input_dim, num_epochs=10, lr=0.01)

# print(f'Best Weight Decay: {best_weight_decay}')

# # Evaluate the final model on the holdout set
# final_model = LogisticRegressionModel(input_dim)
# train_loader = DataLoader(remaining_set, batch_size=64, shuffle=True)
# holdout_loader = DataLoader(holdout_set, batch_size=64, shuffle=False)

# # Train the final model with the best weight decay found
# holdout_f1, _ = train_and_evaluate_model(train_loader, holdout_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=best_weight_decay)
# print(f'Final Model F1 Score on Holdout Set: {holdout_f1:.4f}')

# # Evaluate precision and recall on holdout set
# final_model.eval()
# all_preds = []
# all_labels = []
# with torch.no_grad():
#     for inputs, labels in holdout_loader:
#         outputs = final_model(inputs)
#         predicted = (outputs > 0.5).float()
        
#         all_preds.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

# holdout_precision = precision_score(all_labels, all_preds)
# holdout_recall = recall_score(all_labels, all_preds)

# print(f'Holdout Set Precision: {holdout_precision * 100:.2f}%')
# print(f'Holdout Set Recall: {holdout_recall * 100:.2f}%')

# # Open the file in write mode
# with open('output.txt', 'w') as file:
#     print(f'Best Weight Decay: {best_weight_decay}', file=file)
#     print(f'Final Model F1 Score on Holdout Set: {holdout_f1:.4f}', file=file)
#     print(f'Holdout Set Precision: {holdout_precision * 100:.2f}%', file=file)
#     print(f'Holdout Set Recall: {holdout_recall * 100:.2f}%', file=file)
#     print('Output written to output.txt')

In [3]:
from sympy import *
print_latex(np.logspace(-5, 1, 7))

\mathtt{\text{[1.e-05 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01]}}


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

data1 = pd.read_csv('pseudodata_præoperation.csv')
data2 = pd.read_csv('pseudodata_præoperation.csv')
data3 = pd.read_csv('pseudodata_præoperation.csv')
data4 = pd.read_csv('pseudodata_præoperation.csv')
data5 = pd.read_csv('pseudodata_præoperation.csv')
data6 = pd.read_csv('pseudodata_præoperation.csv')

base = data1
phase1 = pd.concat([data1, data2], axis = 1)
phase2 = pd.concat([data1, data2, data3], axis = 1)
phase3 = pd.concat([data1, data2, data3, data4], axis = 1)
phase4 = pd.concat([data1, data2, data3, data4, data5], axis = 1)
phase5 = pd.concat([data1, data2, data3, data4, data5, data6], axis = 1)

data_list = [base, phase1, phase2, phase3, phase4, phase5]
preds_log = []
models_log = []
holdout_log = []
true_log = []
for i in range(len(data_list)):
    data = data_list[i]
    # Convert to numpy array and generate synthetic labels for demonstration
    X = data.to_numpy()
    np.random.seed(42)
    y = np.random.choice([0, 1], size=len(data))

    # Standardize the data
    X = StandardScaler().fit_transform(X)

    # Convert to PyTorch tensors
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)

    # Create a dataset
    dataset = TensorDataset(X_tensor, y_tensor)

    # Split data into holdout set and remaining set
    train_indices, holdout_indices = train_test_split(np.arange(len(dataset)), test_size=500, random_state=42, stratify=y_tensor.numpy())

    holdout_set = Subset(dataset, holdout_indices)
    remaining_set = Subset(dataset, train_indices)

    y_1_percent = np.count_nonzero(y_tensor) / len(y_tensor)
    y_0_percent = 1 - y_1_percent
    print(f"Class 1 percentage: {y_1_percent * 100:.2f}%")
    print(f"Class 0 percentage: {y_0_percent * 100:.2f}%")
    # Define class weights for BCEWithLogitsLoss
    class_weights = torch.tensor([1/y_0_percent, 1/y_1_percent])

    # Define the logistic regression model with L2 regularization
    class LogisticRegressionModel(nn.Module):
        def __init__(self, input_dim):
            super(LogisticRegressionModel, self).__init__()
            self.linear = nn.Linear(input_dim, 1)
        
        def forward(self, x):
            out = torch.sigmoid(self.linear(x))
            return out

    input_dim = data.shape[1]

    # Function to get target tensor from Subset
    def get_targets(subset):
        return subset.dataset.tensors[1][subset.indices]

    # Get target tensors for holdout and remaining sets
    holdout_targets = get_targets(holdout_set)
    remaining_targets = get_targets(remaining_set)

    # Print class distributions
    print(f"Class distribution in holdout set: {np.unique(holdout_targets.numpy(), return_counts=True)}")
    print(f"Class distribution in remaining set: {np.unique(remaining_targets.numpy(), return_counts=True)}")

    weight_decay_values = np.logspace(-5, 1, 7)  # for example, try values from 1e-5 to 10

    # Function to train and evaluate the model with L2 regularization
    def train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=1e-5):
        model = LogisticRegressionModel(input_dim)
        criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])
        optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
        
        # Training loop
        for epoch in range(num_epochs):
            model.train()
            for inputs, labels in train_loader:
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), labels)
                
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
        # Evaluate on validation set
        model.eval()
        correct = 0
        total = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                predicted = (outputs > 0.5).float().squeeze()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        f1 = f1_score(all_labels, all_preds)
        accuracy = correct / total
        return f1, model, accuracy, all_preds, all_labels

    outer_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    # Function to perform grid search over weight_decay values
    def grid_search(train_set, outer_kf, weight_decays, input_dim, num_epochs=10, lr=0.01):
        best_weight_decay = None
        best_outer_f1 = 0
        y_remaining = y_tensor[train_indices].numpy()

        for weight_decay in weight_decays:
            outer_f1_scores = []
            
            for outer_train_index, outer_test_index in outer_kf.split(np.zeros(len(train_set)), y_remaining):
                
                outer_train_subset = Subset(train_set, outer_train_index)
                outer_test_subset = Subset(train_set, outer_test_index)
                
                # Inner 5-Fold Cross-Validation
                inner_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                inner_f1_scores = []
                
                for inner_train_index, inner_val_index in inner_kf.split(np.zeros(len(outer_train_index)), y_remaining[outer_train_index]):
                    
                    inner_train_indices = np.array(outer_train_index)[inner_train_index]
                    inner_val_indices = np.array(outer_train_index)[inner_val_index]
                    
                    inner_train_subset = Subset(train_set, inner_train_indices)
                    inner_val_subset = Subset(train_set, inner_val_indices)
                    
                    train_loader = DataLoader(inner_train_subset, batch_size=64, shuffle=True)
                    val_loader = DataLoader(inner_val_subset, batch_size=64, shuffle=False)
                    
                    f1, _, _, _, _ = train_and_evaluate_model(train_loader, val_loader, input_dim, num_epochs, lr, weight_decay)
                    inner_f1_scores.append(f1)
                            
                # Train on outer train subset and evaluate on outer test subset
                train_loader = DataLoader(outer_train_subset, batch_size=64, shuffle=True)
                test_loader = DataLoader(outer_test_subset, batch_size=64, shuffle=False)
                
                outer_f1, _, _, _, _ = train_and_evaluate_model(train_loader, test_loader, input_dim, num_epochs, lr, weight_decay)
                outer_f1_scores.append(outer_f1)
            
            mean_outer_f1 = np.mean(outer_f1_scores)
            print(f'Weight Decay: {weight_decay}, Mean Outer F1 Score: {mean_outer_f1:.4f}')
            
            if mean_outer_f1 > best_outer_f1:
                best_outer_f1 = mean_outer_f1
                best_weight_decay = weight_decay
        
        return best_weight_decay


    # Perform grid search over weight_decay values

    best_weight_decay = grid_search(remaining_set, outer_kf, weight_decay_values, input_dim, num_epochs=10, lr=0.01)

    print(f'Best Weight Decay: {best_weight_decay}')

    # Evaluate the final model on the holdout set
    final_model = LogisticRegressionModel(input_dim)
    train_loader = DataLoader(remaining_set, batch_size=64, shuffle=True)
    holdout_loader = DataLoader(holdout_set, batch_size=64, shuffle=False)

    # Train the final model with the best weight decay found
    holdout_f1, _, holdout_acc, holdout_pred, holdout_all_labels = train_and_evaluate_model(train_loader, holdout_loader, input_dim, num_epochs=10, lr=0.01, weight_decay=best_weight_decay)
    print(f'Final Model F1 Score on Holdout Set: {holdout_f1:.4f}')
    print(f'Holdout Set Accuracy: {holdout_acc * 100:.2f}%')
    torch.save(final_model.state_dict(), 'path_to_my_model.pth')
    models_log.append(final_model)
    holdout_log.append(holdout_loader)
    true_log.append(holdout_all_labels)
    preds_log.append(holdout_pred)

    # Evaluate precision and recall on holdout set
    final_model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in holdout_loader:
            outputs = final_model(inputs)
            predicted = (outputs > 0.5).float()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    holdout_precision = precision_score(all_labels, all_preds)
    holdout_recall = recall_score(all_labels, all_preds)

    print(f'Holdout Set Precision: {holdout_precision * 100:.2f}%')
    print(f'Holdout Set Recall: {holdout_recall * 100:.2f}%')

    # Open the file in write mode
    with open('output.txt', 'a') as file:
        print(f"Phase {i + 1}", file=file)
        print(f'Best Weight Decay: {best_weight_decay}', file=file)
        print(f'Final Model F1 Score on Holdout Set: {holdout_f1:.4f}', file=file)
        print(f'Holdout Set Accuracy: {holdout_acc * 100:.2f}%', file=file)
        print(f'Holdout Set Precision: {holdout_precision * 100:.2f}%', file=file)
        print(f'Holdout Set Recall: {holdout_recall * 100:.2f}%', file=file)
        print('\n', file=file)
        print('Output written to output.txt')


Class 1 percentage: 49.39%
Class 0 percentage: 50.61%
Class distribution in holdout set: (array([0., 1.], dtype=float32), array([253, 247]))
Class distribution in remaining set: (array([0., 1.], dtype=float32), array([1359, 1326]))
Weight Decay: 1e-05, Mean Outer F1 Score: 0.5044
Weight Decay: 0.0001, Mean Outer F1 Score: 0.5180
Weight Decay: 0.001, Mean Outer F1 Score: 0.5032
Weight Decay: 0.01, Mean Outer F1 Score: 0.5028
Weight Decay: 0.1, Mean Outer F1 Score: 0.5195
Weight Decay: 1.0, Mean Outer F1 Score: 0.5296
Weight Decay: 10.0, Mean Outer F1 Score: 0.5200
Best Weight Decay: 1.0
Final Model F1 Score on Holdout Set: 0.5481
Holdout Set Accuracy: 51.20%
Holdout Set Precision: 48.05%
Holdout Set Recall: 49.80%
Output written to output.txt
Class 1 percentage: 49.39%
Class 0 percentage: 50.61%
Class distribution in holdout set: (array([0., 1.], dtype=float32), array([253, 247]))
Class distribution in remaining set: (array([0., 1.], dtype=float32), array([1359, 1326]))
Weight Decay: 1e

KeyboardInterrupt: 

In [ ]:
def compute_mcnemar_contingency(true_labels, pred_A, pred_B):
    assert len(true_labels) == len(pred_A) == len(pred_B), "Inputs must have the same length"
    
    n00 = 0  # Number of items misclassified by both A and B
    n01 = 0  # Number of items misclassified by A but not by B
    n10 = 0  # Number of items misclassified by B but not by A
    n11 = 0  # Number of items classified correctly by both A and B
    
    for true_label, pred_a, pred_b in zip(true_labels, pred_A, pred_B):
        if pred_a != true_label and pred_b != true_label:
            n00 += 1
        elif pred_a != true_label and pred_b == true_label:
            n01 += 1
        elif pred_a == true_label and pred_b != true_label:
            n10 += 1
        elif pred_a == true_label and pred_b == true_label:
            n11 += 1
    
    return n00, n01, n10, n11

n00, n01, n10, n11 = compute_mcnemar_contingency(true_log[0], preds_log[0], preds_log[1])

print(f"n00: {n00}")
print(f"n01: {n01}")
print(f"n10: {n10}")
print(f"n11: {n11}")

cont_table_pred_set = [[n00, n01], [n10, n11]]
print("Contingency Table for Predictions:")
print(cont_table_pred_set)

In [ ]:
# Import library
from statsmodels.stats.contingency_tables import mcnemar

threshold = 3.841 
significance_value = 0.05

# McNemar's Test with the continuity correction
test = mcnemar(cont_table_pred_set, exact=False, correction=True)

if test.pvalue < significance_value:
  print("Reject Null hypotesis")
else:
  print("Fail to reject Null hypotesis")

#or equivalently
if test.statistic > threshold:
  print("Reject Null hypotesis")
else:
  print("Fail to reject Null hypotesis")

In [ ]:
models_log

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

def evaluate_roc_auc(models, loaders):
    plt.figure(figsize=(8, 6))

    for i, (model, loader) in enumerate(zip(models, loaders)):
        model.eval()
        y_true = []
        y_scores = []

        with torch.no_grad():
            for images, labels in loader:
                outputs = model(images)
                y_true.extend(labels.numpy())
                y_scores.extend(outputs.numpy())

        fpr, tpr, thresholds = roc_curve(y_true, y_scores)
        auc_score = roc_auc_score(y_true, y_scores)

        plt.plot(fpr, tpr, label=f'Model {i} ROC curve (area = {auc_score:.2f})')
    
    plt.plot([0, 1], [0, 1], color='grey', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

# Example usage
# Assuming model0 and model1 are your models and holdout_loader0 and holdout_loader1 are the corresponding loaders
evaluate_roc_auc(models_log, holdout_log)